In [1]:
import pydeck as pdk
import pandas as pd
import numpy as np
import json

# Exploring Phantom Rides

In [2]:
phantom = pd.read_csv('./../data/phantom2.csv')

In [3]:
phantom.head(2)

,bike_id,end_station_id,end_station_name,end_lat,end_lon,end_time,next_station_id,next_station_name,next_lat,next_lon,next_time,end_borough,next_borough,end_day,next_day
0,16344,507,E 25 St & 2 Ave,40.739126,-73.979738,2013-06-09 20:40:52,164,E 47 St & 2 Ave,40.753231,-73.970325,2013-06-10 09:49:57,Manhattan,Manhattan,1,2
1,16344,510,W 51 St & 6 Ave,40.760660,-73.980420,2013-06-12 07:53:02,484,W 44 St & 5 Ave,40.755003,-73.980144,2013-06-13 17:29:33,Manhattan,Manhattan,4,5


# Weekday Rebalancing Rides


In [77]:
weekday_phantom = phantom[phantom['end_day'].isin([2,3,4,5,6])]

In [78]:
weekday_grouped = weekday_phantom.groupby(['end_station_id','next_station_id'])\
    .agg(count=('bike_id',np.size),
        end_station = ('end_station_name',np.min),
        end_lat = ('end_lat', np.min),
        end_lon = ('end_lon', np.min),
        next_station = ('next_station_name', np.min),
        next_lat = ('next_lat',np.min),
        next_lon = ('next_lon', np.min),
        ).reset_index()

In [79]:
weekday_grouped.sort_values(by = 'count',ascending = False, inplace = True)

In [111]:
RED_RGB = [217, 38, 28, 100]
GREEN_RGB = [0, 255, 0, 100]
arc_layer = pdk.Layer(
    "ArcLayer",
    data = weekday_grouped[:300],
    get_source_position=['end_lon','end_lat'],
    get_target_position = ['next_lon','next_lat'],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
    get_width = 'count/700')

view_state = pdk.ViewState(latitude = 40.74,longitude = -74.0, bearing=290, pitch=50, zoom=12,)
# view_state = pdk.ViewState(latitude = 40.778786,longitude = -73.974785, bearing=45, pitch=50, zoom=11,)


TOOLTIP_TEXT = {"html": "{count} bikes moved from {end_station}<br />\
                to {next_station}"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT,map_style = 'light')
r.to_html("weekday_rebalancing.html")
# with open('weekday_rebalancing.json', 'w') as outfile:
#     json.dump(r.to_json(), outfile)


# Weekend Rebalancing Rides

In [102]:
weekend_phantom = phantom[phantom['end_day'].isin([1,7])]

In [103]:
weekend_grouped = weekend_phantom.groupby(['end_station_id','next_station_id'])\
    .agg(count=('bike_id',np.size),
        end_station = ('end_station_name',np.min),
        end_lat = ('end_lat', np.min),
        end_lon = ('end_lon', np.min),
        next_station = ('next_station_name', np.min),
        next_lat = ('next_lat',np.min),
        next_lon = ('next_lon', np.min),
        ).reset_index()

In [104]:
weekend_grouped.sort_values(by = 'count',ascending = False, inplace = True)

In [110]:
RED_RGB = [217, 38, 28, 100]
GREEN_RGB = [0, 255, 0, 100]
arc_layer = pdk.Layer(
    "ArcLayer",
    data = weekend_grouped[:300],
    get_source_position=['end_lon','end_lat'],
    get_target_position = ['next_lon','next_lat'],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
    get_width = 'count/100')

view_state = pdk.ViewState(latitude = 40.74,longitude = -74.0, bearing=290, pitch=50, zoom=12,)
# latitude = 40.778786,longitude = -73.974785

TOOLTIP_TEXT = {"html": "{count} bikes moved from {end_station}<br />\
                to {next_station}"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT,map_style = 'light')
r.to_html("weekend_rebalancing.html")
with open('weekend_rebalancing.json', 'w') as outfile:
    json.dump(r.to_json(), outfile)

# Lets get a bit more refined
We will start by lookig at the rebalancing that happens in the AM of the weedays

In [19]:
phantom3 = pd.read_parquet('./../data/phantom3.parquet')

In [52]:
weekday_morning = phantom3[(phantom3['end_day'] == phantom3['next_day']) &\
                           phantom3['midpoint'].dt.dayofweek.isin([0,1,2,3,4]) &\
                          phantom3['midpoint'].dt.hour.isin([])]

In [53]:
weekday_morning = weekday_morning.groupby(['end_station_id','next_station_id'])\
    .agg(count=('bike_id',np.size),
        end_station = ('end_station_name',np.min),
        end_lat = ('end_lat', np.min),
        end_lon = ('end_lon', np.min),
        next_station = ('next_station_name', np.min),
        next_lat = ('next_lat',np.min),
        next_lon = ('next_lon', np.min),
        ).reset_index()

In [92]:
weekday_morning.sort_values(by = 'count', ascending = False,inplace = True)

In [93]:
BLUE_RGB = [0, 59, 112, 90]
RED_RGB = [217, 38, 28, 90]
arc_layer = pdk.Layer(
    "ArcLayer",
    data = weekday_morning[:300],
    get_source_position=['end_lon','end_lat'],
    get_target_position = ['next_lon','next_lat'],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=BLUE_RGB,
    pickable=True,
    auto_highlight=True,
    get_width = 'count/2')

view_state = pdk.ViewState(latitude = 40.778786,longitude = -73.974785, bearing=45, pitch=50, zoom=11,)
# latitude = 40.778786,longitude = -73.974785

TOOLTIP_TEXT = {"html": "{count} bikes moved from {end_station}<br />\
                to {next_station}Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT,map_style = 'light')
r.to_html("weekend_rebalancing.html")

# Just  May 1st 2018-  April 30th 2019 Weekday

In [54]:
year = phantom3[(phantom3['end_time'] >= pd.to_datetime('2018-05-01 00:00:00')) &\
         (phantom3['next_time'] < pd.to_datetime('2019-05-01 00:00:00'))]

In [55]:
weekday_year = year[year['midpoint'].dt.dayofweek.isin([0,1,2,3,4])]

In [56]:
weekday_year.head(10)

,bike_id,end_station_id,end_station_name,end_lat,end_lon,end_time,next_station_id,next_station_name,next_lat,next_lon,next_time,end_borough,next_borough,end_day,next_day,midpoint
491,16344,3168,Central Park West & W 85 St,40.784727,-73.969617,2018-05-08 23:26:15.456,3160,Central Park West & W 76 St,40.778968,-73.973747,2018-05-09 08:43:58.007,Manhattan,Manhattan,3,4,2018-05-09 04:05:15.456
492,16344,433,E 13 St & Avenue A,40.729554,-73.980572,2018-05-17 20:26:16.744,497,E 17 St & Broadway,40.737050,-73.990093,2018-05-17 20:45:05.722,Manhattan,Manhattan,5,5,2018-05-17 20:36:16.744
495,16344,492,W 33 St & 7 Ave,40.750200,-73.990931,2018-06-27 15:28:24.803,3341,Central Park West & W 102 St,40.795346,-73.961860,2018-08-22 17:48:51.736,Manhattan,Manhattan,4,4,2018-07-25 16:38:24.803
496,16344,359,E 47 St & Park Ave,40.755103,-73.974987,2018-08-23 08:48:17.577,3167,Amsterdam Ave & W 73 St,40.779668,-73.980930,2018-08-24 07:59:31.511,Manhattan,Manhattan,5,6,2018-08-23 20:24:17.577
497,16344,3374,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955613,2018-09-04 20:51:23.645,3164,Columbus Ave & W 72 St,40.777057,-73.978985,2018-09-05 09:14:41.063,Manhattan,Manhattan,3,4,2018-09-05 03:03:23.645
498,16344,3318,2 Ave & E 96 St,40.783964,-73.947167,2018-09-05 19:18:49.427,3552,W 113 St & Broadway,40.805973,-73.964928,2018-09-06 08:22:54.477,Manhattan,Manhattan,4,5,2018-09-06 01:50:49.427
499,16344,3412,Pacific St & Nevins St,40.685376,-73.983021,2018-09-11 20:02:12.460,3324,3 Ave & 14 St,40.668546,-73.993333,2018-09-15 01:46:40.108,Brooklyn,Brooklyn,3,7,2018-09-13 10:54:12.460
500,16344,491,E 24 St & Park Ave S,40.740964,-73.986022,2018-09-21 05:55:46.277,537,Lexington Ave & E 24 St,40.740259,-73.984092,2018-09-21 08:00:14.466,Manhattan,Manhattan,6,6,2018-09-21 06:58:46.277
502,16344,267,Broadway & W 36 St,40.750977,-73.987654,2018-10-08 15:32:30.978,362,Broadway & W 37 St,40.751726,-73.987535,2018-10-08 17:07:31.831,Manhattan,Manhattan,2,2,2018-10-08 16:20:30.978
503,16344,356,Bialystoker Pl & Delancey St,40.716226,-73.982612,2018-10-09 18:59:17.815,3164,Columbus Ave & W 72 St,40.777057,-73.978985,2018-10-10 09:23:25.222,Manhattan,Manhattan,3,4,2018-10-10 02:11:17.815


In [57]:
weekday_year = weekday_year.groupby(['end_station_id','next_station_id'])\
    .agg(count=('bike_id',np.size),
        end_station = ('end_station_name',np.min),
        end_lat = ('end_lat', np.min),
        end_lon = ('end_lon', np.min),
        next_station = ('next_station_name', np.min),
        next_lat = ('next_lat',np.min),
        next_lon = ('next_lon', np.min),
        ).reset_index()
weekday_year.sort_values(by = 'count', ascending = False,inplace = True)

In [58]:
BLUE_RGB = [0, 59, 112, 90]
RED_RGB = [217, 38, 28, 90]
arc_layer = pdk.Layer(
    "ArcLayer",
    data = weekday_year[:300],
    get_source_position=['end_lon','end_lat'],
    get_target_position = ['next_lon','next_lat'],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=BLUE_RGB,
    pickable=True,
    auto_highlight=True,
    get_width = 'count/150')

view_state = pdk.ViewState(latitude = 40.778786,longitude = -73.974785, bearing=45, pitch=50, zoom=11,)
# latitude = 40.778786,longitude = -73.974785

TOOLTIP_TEXT = {"html": "{count} bikes moved from {end_station}<br />\
                to {next_station}Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT,map_style = 'light')
r.to_html("whatever.html")

# Now the WEekend

In [59]:
weekend_year = year[year['midpoint'].dt.dayofweek.isin([5,6])]

In [60]:
weekend_year = weekend_year.groupby(['end_station_id','next_station_id'])\
    .agg(count=('bike_id',np.size),
        end_station = ('end_station_name',np.min),
        end_lat = ('end_lat', np.min),
        end_lon = ('end_lon', np.min),
        next_station = ('next_station_name', np.min),
        next_lat = ('next_lat',np.min),
        next_lon = ('next_lon', np.min),
        ).reset_index()
weekend_year.sort_values(by = 'count', ascending = False,inplace = True)

In [62]:
BLUE_RGB = [0, 59, 112, 90]
RED_RGB = [217, 38, 28, 90]
arc_layer = pdk.Layer(
    "ArcLayer",
    data = weekend_year[:300],
    get_source_position=['end_lon','end_lat'],
    get_target_position = ['next_lon','next_lat'],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=BLUE_RGB,
    pickable=True,
    auto_highlight=True,
    get_width = 'count/50')

view_state = pdk.ViewState(latitude = 40.778786,longitude = -73.974785, bearing=45, pitch=50, zoom=11,)
# latitude = 40.778786,longitude = -73.974785

TOOLTIP_TEXT = {"html": "{count} bikes moved from {end_station}<br />\
                to {next_station}Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT,map_style = 'light')
r.to_html("whatever2.html")